In [1]:
#
#Tweets Machine Learning Coursework 
#COMP3222
#

# To support both python 2 and 3
from __future__ import division, print_function, unicode_literals

# Common imports
import os
import numpy as np
import pandas as pd

# These two lines are required to use Tensorflow 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# To plot nice figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Clear tensorflow's and reset seed
def reset_graph(seed=None):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)



Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#get data
def getData(file_name):
    dict_data = {}
    data_file = open(file_name, "r", encoding="utf8")
    raw_data_txt = data_file.readlines()
    raw_data = []
    for data in raw_data_txt:
        raw_data.append(data.split("\t"))
    raw_data.pop(0)

    target_text = [data[6] for data in raw_data]
    dict_data['text'] = np.array([tweet[1] for tweet in raw_data])
    dict_data['imageIds'] = np.array([tweet[3] for tweet in raw_data])
    dict_data['timestamp'] = np.array([tweet[5] for tweet in raw_data])
    dict_data['label'] = np.array([1 if target=='real' or target =='real\n' else 0 for target in target_text])

    return dict_data

df_train = pd.DataFrame.from_dict(getData("mediaeval-2015-trainingset.txt"))
df_test = pd.DataFrame.from_dict(getData("mediaeval-2015-testset.txt"))
print("Done")


Done


In [36]:
#save file pre-processing to csv
#df_test.to_csv('test_v3.csv')

In [3]:
df_train = pd.read_csv("train_v4.csv")
df_train = df_train.iloc[:, 1:]
df_train

text        imageIds  \
0      remember movie Day Tomorrow reminds happening ...  sandyA_fake_46   
1      Look Sandy NY Tremendous image hurricane Looks...  sandyA_fake_09   
2      Good photo Hurricane Sandy reminds movie Indep...  sandyA_fake_09   
3                                Scary shit hurricane NY  sandyA_fake_29   
4      fave place world nyc hurricane sandy statueofl...  sandyA_fake_15   
...                                                  ...             ...   
14478         slaps TweetDeck PigFish http co pyHcJn0jwA      pigFish_01   
14479  New Species Fish found Brazil Really good Phot...      pigFish_01   
14480                    call pigFISH http co 4Bml62OD15      pigFish_01   
14481         Pigfish shark pork fish http co hQzWGhyDef      pigFish_01   
14482        decide fish meat Pigfish http co 5JBtF54cmg      pigFish_01   

                              timestamp language  significant_fake  \
0        Mon Oct 29 22:34:01 +0000 2012       es                 1   
1        Mon Oct 29 19:11:23 +0000 2012       es                 1   
2        Mon Oct 29 18:11:08 +0000 2012       es                 1   
3        Mon Oct 29 19:15:33 +0000 2012       en                 1   
4        Mon Oct 29 20:46:02 +0000 2012       en                 1   
...                                 ...      ...               ...   
14478  Tue Mar 11 03: 48: 36 +0000 2014       en                 0   
14479  Mon Mar 10 18: 09: 26 +0000 2014       en                 1   
14480  Mon Mar 10 10: 59: 45 +0000 2014       en                 0   
14481  Sun Mar 09 20: 07: 10 +0000 2014       it                 1   
14482  Sun Mar 09 16: 36: 09 +0000 2014       en                 0   

       significant_real  misspellings  news_company_found  num_emoji  \
0                     0             0                   0          0   
1                     0             1                   0          0   
2                     0             1                   0          0   
3                     0             0                   0          0   
4                     0             2                   0          1   
...                 ...           ...                 ...        ...   
14478                 0             4                   0          0   
14479                 0             2                   0          0   
14480                 0             3                   0          0   
14481                 0             3                   0          0   
14482                 0             3                   0          0   

       num_hashtags  profane  retweet  label  
0                 1        0        0      0  
1                 0        0        1      0  
2                 2        0        0      0  
3                 2        1        0      0  
4                 4        0        0      0  
...             ...      ...      ...    ...  
14478             0        0        0      0  
14479             0        0        0      0  
14480             1        0        0      0  
14481             0        0        0      0  
14482             1        0        0      0  

[14483 rows x 13 columns]

In [22]:
df_test = pd.read_csv("test_v1.csv")
df_test = df_test.iloc[:, 1:]
df_test

text      imageIds  \
0                                kereeen RT Eclipse ISS   eclipse_01    
1                   Absolutely beautiful RT Eclipse ISS   eclipse_01    
2                          Eclipse ISS 3 20 Wow amazing   eclipse_01    
3                                           Eclipse ISS   eclipse_01    
4     Eclipse seen ISS Something else Divine creatio...    eclipse_01   
...                                                 ...           ...   
3776  ZDF presenter confesses rigged video Varoufaki...  varoufakis_1   
3777  Oh kleine liars ZDF presenter confesses faked ...  varoufakis_1   
3778       ZDF program confirm Varoufakis video montage  varoufakis_1   
3779  11 34 ALMOST noon Big confusion Varoufakis vid...  varoufakis_1   
3780  Sorry ENGLISH subtitles full English Greek sub...  varoufakis_1   

                           timestamp language  profane  misspellings  \
0     Fri Mar 20 09:45:43 +0000 2015       en        0             2   
1     Fri Mar 20 11:04:02 +0000 2015       en        0             1   
2     Fri Mar 20 12:10:06 +0000 2015       en        0             0   
3     Fri Mar 20 09:12:41 +0000 2015       en        0             0   
4     Fri Mar 20 17:44:11 +0000 2015       fr        0             0   
...                              ...      ...      ...           ...   
3776  Thu Mar 19 05:49:44 +0000 2015       fr        0             2   
3777  Thu Mar 19 05:51:42 +0000 2015       fr        0             2   
3778  Thu Mar 19 09:23:21 +0000 2015       es        0             2   
3779  Thu Mar 19 10:35:20 +0000 2015       de        0             4   
3780  Wed Mar 18 21:21:05 +0000 2015       en        0             1   

      num_emoji  num_hashtags  news_company_found  retweet  label  
0             0             0                   0        1      0  
1             0             0                   0        1      0  
2             0             0                   0        0      0  
3             0             0                   0        0      0  
4             1             0                   0        0      0  
...         ...           ...                 ...      ...    ...  
3776          0             1                   0        0      0  
3777          0             0                   0        0      0  
3778          0             0                   0        0      0  
3779          2             0                   0        0      0  
3780          0             1                   0        0      0  

[3781 rows x 11 columns]

In [5]:
#Find all the different languages
#https://meta.wikimedia.org/wiki/Template:List_of_language_names_ordered_by_code conversion of symbol to langauge table
from langdetect import detect

def language_detect(data_frame):
    languages = []
    for tweet in data_frame['text'].values:
        try:
            languages.append(detect(tweet))
        except:
            languages.append('en')

    data_frame.insert(3, "language", languages, True)

    languages_seen = set(languages)
    print("Languages seen: ", languages_seen)
    return data_frame

In [6]:
def retweet_detection(data_frame):
    retweets = [1 if tweet.count("RT") >= 1 else 0 for tweet in data_frame["text"].values]
    data_frame.insert(4, "retweet", retweets, True)
    return data_frame

In [8]:
from deep_translator import GoogleTranslator

def translate(data_frame):
    translate_text = {}
    for i in range(len(data_frame)):
        if(data_frame.iloc[i]['language'] != 'en'):
            translate_text[data_frame.iloc[i]['text']] = GoogleTranslator(source='auto', target='en').translate(data_frame.iloc[i]['text'])
            #print(i, data_frame.iloc[i]['language'], GoogleTranslator(source='auto', target='en').translate(df_train.iloc[i]['text']))

    data_frame["text"].replace(translate_text, inplace=True)
    return data_frame

In [9]:
df_train

text        imageIds  \
0      remember movie Day Tomorrow reminds happening ...  sandyA_fake_46   
1      Look Sandy NY Tremendous image hurricane Looks...  sandyA_fake_09   
2      Good photo Hurricane Sandy reminds movie Indep...  sandyA_fake_09   
3                                Scary shit hurricane NY  sandyA_fake_29   
4      fave place world nyc hurricane sandy statueofl...  sandyA_fake_15   
...                                                  ...             ...   
14478         slaps TweetDeck PigFish http co pyHcJn0jwA      pigFish_01   
14479  New Species Fish found Brazil Really good Phot...      pigFish_01   
14480                    call pigFISH http co 4Bml62OD15      pigFish_01   
14481         Pigfish shark pork fish http co hQzWGhyDef      pigFish_01   
14482        decide fish meat Pigfish http co 5JBtF54cmg      pigFish_01   

                              timestamp language  significant_fake  \
0        Mon Oct 29 22:34:01 +0000 2012       es                 1   
1        Mon Oct 29 19:11:23 +0000 2012       es                 1   
2        Mon Oct 29 18:11:08 +0000 2012       es                 1   
3        Mon Oct 29 19:15:33 +0000 2012       en                 1   
4        Mon Oct 29 20:46:02 +0000 2012       en                 1   
...                                 ...      ...               ...   
14478  Tue Mar 11 03: 48: 36 +0000 2014       en                 0   
14479  Mon Mar 10 18: 09: 26 +0000 2014       en                 1   
14480  Mon Mar 10 10: 59: 45 +0000 2014       en                 0   
14481  Sun Mar 09 20: 07: 10 +0000 2014       it                 1   
14482  Sun Mar 09 16: 36: 09 +0000 2014       en                 0   

       significant_real  misspellings  news_company_found  num_emoji  \
0                     0             0                   0          0   
1                     0             1                   0          0   
2                     0             1                   0          0   
3                     0             0                   0          0   
4                     0             2                   0          1   
...                 ...           ...                 ...        ...   
14478                 0             4                   0          0   
14479                 0             2                   0          0   
14480                 0             3                   0          0   
14481                 0             3                   0          0   
14482                 0             3                   0          0   

       num_hashtags  profane  retweet  label  
0                 1        0        0      0  
1                 0        0        1      0  
2                 2        0        0      0  
3                 2        1        0      0  
4                 4        0        0      0  
...             ...      ...      ...    ...  
14478             0        0        0      0  
14479             0        0        0      0  
14480             1        0        0      0  
14481             0        0        0      0  
14482             1        0        0      0  

[14483 rows x 13 columns]

In [10]:
import spacy
from profanity_filter import ProfanityFilter

def profanity_detection(data_frame):
    nlp = spacy.load('en_core_web_sm')
    profanity_filter = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
    nlp.add_pipe(profanity_filter.spacy_component, last=True)

    doc = nlp('This is fuck shit!')

    doc._.is_profane
    profanity_check = []
    count = 0
    for tweet in data_frame['text'].values:
        text = nlp(tweet)
        if(text._.is_profane):
            profanity_check.append(1)
        else: 
            profanity_check.append(0)
        
        #print(count, text._.is_profane)
        count += 1
    
    data_frame.insert(4, "profane", profanity_check, True)
    return data_frame

In [11]:
def numofhashtags(data_frame):
    num_hashtags = []
    for i in range(len(data_frame)):
        num_hashtags.append(data_frame.iloc[i]['text'].count("#"))
    #maybe count number of punctuation as well.
    data_frame.insert(4, "num_hashtags", num_hashtags, True)
    return data_frame


In [12]:
import emoji
def emoji_detect(data_frame):
    num_emojis = []
    for i in range(len(data_frame)):
        emoji_count = emoji.emoji_count(data_frame.iloc[i]['text'])
        num_emojis.append(emoji_count)
    
    data_frame.insert(4, "num_emoji", num_emojis, True)
    return data_frame


In [13]:
import regex as re

def parseText(data_frame):
    parsed_text = {}
    for i in range(len(data_frame)):
        parsed = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",data_frame.iloc[i]['text']).split())
        parsed_text[data_frame.iloc[i]['text']] = parsed
        #print(i, parsed)
    data_frame["text"].replace(parsed_text, inplace=True)
    return data_frame
    

In [14]:
#based of https://craft.co/cnn/competitors
#
def new_companies(data_frame):
    news_companies =["cnn", "bbc", "nbc", "new york times", "wall street journal", " ap ", "fox", "cnbc", "daily mail"]
    news_found = []
    for tweet in data_frame['text'].values:
        tweet = tweet.lower()
        if(any(news in tweet for news in news_companies) or (tweet.find('ap') == 0) or (tweet.find('ap') == len(tweet)-2) ) :
            news_found.append(1)
        else:
            news_found.append(0)
    data_frame.insert(4, "news_company_found", news_found, True)
    return data_frame

In [15]:
#location detector too much noise picked up

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="COMP322-MLT")
x = "My fave place in the world nyc hurricane sandy times sqaure"
x = x.split(' ')
for i in range(len(x)):
    try:
        location = geolocator.geocode(x[i])
        print(location.address)
    except:
        print("Notfound")

United States
Fave, Jalan Cihampelas, Cipaganti, Coblong, Bandung, Jawa Barat, 40131, Indonesia
Placé, Mayenne, Pays de la Loire, France métropolitaine, 53240, France
India
The (Oval) Banqueting Suite, 372, Stratford Road, Moseley, Sparkhill, Birmingham, West Midlands Combined Authority, West Midlands, England, B11 4AB, United Kingdom
World, 어울마당로, 홍대거리, 서교동, 마포구, 서울, 04039, 대한민국
New York, United States
Hurricane, City Of Karratha, Western Australia, Australia
Sandy, Salt Lake County, Utah, United States
Кожай-Семёновка, Кожай-Семеновский сельсовет, Миякинский район, Башкортостан, Приволжский федеральный округ, Россия
Barkman Sqaure, Midland, Midland County, Texas, 79705, United States


In [16]:
#Run hashtags if first time running on machine

from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

def stopWordRemoval(data_frame):
    stop_words = set(stopwords.words('english'))
    filtered_text = {}
    for i in range(len(data_frame)):
        word_tokens = word_tokenize(data_frame.iloc[i]['text'])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        new_sentence = " ".join(filtered_sentence)
        filtered_text[data_frame.iloc[i]['text']] = new_sentence
        #print(i, new_sentence)
    data_frame["text"].replace(filtered_text, inplace=True)
    return data_frame

In [17]:
from spellchecker import SpellChecker
def misspelling_detection(data_frame):
    spell = SpellChecker()
    misspelled_count = []
    for i in range(len(data_frame)):
        word_list = word_tokenize(data_frame.iloc[i]['text'])
        misspelled = spell.unknown(word_list)
        misspelled_count.append(len(misspelled))
        #print(i, len(misspelled), misspelled))
    data_frame.insert(4, "misspellings", misspelled_count, True)
    return data_frame

In [19]:
from collections import defaultdict, Counter

fake_words = defaultdict(lambda: 0)
real_words = defaultdict(lambda: 0)
total_words = defaultdict(lambda: 0)

for i in range(len(df_train)):
    text = str(df_train.iloc[i]['text']).lower()
    text = text.split()
    if(df_train.iloc[i]['label'] == 0):
        for j in range(len(text)):
            fake_words[text[j]] += 1
            total_words[text[j]] += 1
    else:
        for j in range(len(text)):
            real_words[text[j]] += 1
            total_words[text[j]] += 1


real_words = dict(sorted(real_words.items(), reverse=True, key=lambda item: item[1]))
fake_words = dict(sorted(fake_words.items(), reverse=True, key=lambda item: item[1]))
total_words = dict(sorted(total_words.items(), reverse=False, key=lambda item: item[1]))

words = list(total_words.keys())
for w in words:
    if(total_words[w] < 100):
        del total_words[w]
        try:
            del real_words[w]
        except:
            pass
        try:
            del fake_words[w]
        except:
            pass
    
rw = Counter(real_words)
fw = Counter(fake_words)
fake_res = fw - rw
real_res = rw - fw

real_res = {k: real_res[k]/total_words[k] for k in total_words.keys() & real_res}
fake_res = {k: fake_res[k]/total_words[k] for k in total_words.keys() & fake_res}

real_res = dict(sorted(real_res.items(), reverse=True, key=lambda item: item[1]))
fake_res = dict(sorted(fake_res.items(), reverse=True, key=lambda item: item[1]))


sig_real_words = list(k for k, v in real_res.items() if v > 0.5)
sig_fake_words = list(k for k, v in fake_res.items() if v > 0.5)

def significance(data_frame, sig_words, clmn_name):
    sig_found = []
    for tweet in data_frame['text'].values:
        try:
            tweet = tweet.lower()
            if(any(word in tweet for word in sig_words)):
                sig_found.append(1)
            else:
                sig_found.append(0)
        except:
            sig_found.append(0)
    data_frame.insert(4, clmn_name, sig_found, True)
    return data_frame

In [55]:
df_test = language_detect(df_test)
print("Language Detection Done")
df_test = translate(df_test)
print("Translation Done")
df_test = retweet_detection(df_test)
print("Retweet Detection Done")
df_test = new_companies(df_test)
print("News companies Detection Done")
df_test = numofhashtags(df_test)
print("Hashtag count Done")
df_test = emoji_detect(df_test)
print("Emoji count Done")
df_test = parseText(df_test)
print("Parsing Done")
df_test = stopWordRemoval(df_test)
print("Stopwprd removal Done")
df_test = misspelling_detection(df_test)
print("Misspelling Count Done")
df_test = profanity_detection(df_test)
print("Profanity Detection Done")

print("Preprocessing done")
#before passing the text as unigram .lower everything

Languages seen:  {'ta', 'sq', 'lt', 'el', 'ar', 'so', 'tl', 'pl', 'hi', 'de', 'ro', 'ru', 'th', 'fr', 'ko', 'es', 'ca', 'en', 'ja', 'pt', 'af', 'id', 'cy', 'nl', 'et', 'it', 'fi', 'sv', 'te', 'vi', 'tr', 'bg', 'hr'}
Language Detection Done
Translation Done
Retweet Detection Done
News companies Detection Done
Hashtag count Done
Emoji count Done
Parsing Done
Stopwprd removal Done
Misspelling Count Done
Profanity Detection Done
Preprocessing done


In [23]:
df_test = significance(df_test, sig_real_words, "significant_real")
df_test = significance(df_test, sig_fake_words, "significant_fake")
df_test

text      imageIds  \
0                                kereeen RT Eclipse ISS   eclipse_01    
1                   Absolutely beautiful RT Eclipse ISS   eclipse_01    
2                          Eclipse ISS 3 20 Wow amazing   eclipse_01    
3                                           Eclipse ISS   eclipse_01    
4     Eclipse seen ISS Something else Divine creatio...    eclipse_01   
...                                                 ...           ...   
3776  ZDF presenter confesses rigged video Varoufaki...  varoufakis_1   
3777  Oh kleine liars ZDF presenter confesses faked ...  varoufakis_1   
3778       ZDF program confirm Varoufakis video montage  varoufakis_1   
3779  11 34 ALMOST noon Big confusion Varoufakis vid...  varoufakis_1   
3780  Sorry ENGLISH subtitles full English Greek sub...  varoufakis_1   

                           timestamp language  significant_fake  \
0     Fri Mar 20 09:45:43 +0000 2015       en                 0   
1     Fri Mar 20 11:04:02 +0000 2015       en                 1   
2     Fri Mar 20 12:10:06 +0000 2015       en                 0   
3     Fri Mar 20 09:12:41 +0000 2015       en                 0   
4     Fri Mar 20 17:44:11 +0000 2015       fr                 0   
...                              ...      ...               ...   
3776  Thu Mar 19 05:49:44 +0000 2015       fr                 1   
3777  Thu Mar 19 05:51:42 +0000 2015       fr                 1   
3778  Thu Mar 19 09:23:21 +0000 2015       es                 0   
3779  Thu Mar 19 10:35:20 +0000 2015       de                 0   
3780  Wed Mar 18 21:21:05 +0000 2015       en                 1   

      significant_real  profane  misspellings  num_emoji  num_hashtags  \
0                    0        0             2          0             0   
1                    0        0             1          0             0   
2                    0        0             0          0             0   
3                    0        0             0          0             0   
4                    0        0             0          1             0   
...                ...      ...           ...        ...           ...   
3776                 0        0             2          0             1   
3777                 0        0             2          0             0   
3778                 0        0             2          0             0   
3779                 1        0             4          2             0   
3780                 0        0             1          0             1   

      news_company_found  retweet  label  
0                      0        1      0  
1                      0        1      0  
2                      0        0      0  
3                      0        0      0  
4                      0        0      0  
...                  ...      ...    ...  
3776                   0        0      0  
3777                   0        0      0  
3778                   0        0      0  
3779                   0        0      0  
3780                   0        0      0  

[3781 rows x 13 columns]

In [21]:
df_train

text        imageIds  \
0      remember movie Day Tomorrow reminds happening ...  sandyA_fake_46   
1      Look Sandy NY Tremendous image hurricane Looks...  sandyA_fake_09   
2      Good photo Hurricane Sandy reminds movie Indep...  sandyA_fake_09   
3                                Scary shit hurricane NY  sandyA_fake_29   
4      fave place world nyc hurricane sandy statueofl...  sandyA_fake_15   
...                                                  ...             ...   
14478         slaps TweetDeck PigFish http co pyHcJn0jwA      pigFish_01   
14479  New Species Fish found Brazil Really good Phot...      pigFish_01   
14480                    call pigFISH http co 4Bml62OD15      pigFish_01   
14481         Pigfish shark pork fish http co hQzWGhyDef      pigFish_01   
14482        decide fish meat Pigfish http co 5JBtF54cmg      pigFish_01   

                              timestamp language  significant_fake  \
0        Mon Oct 29 22:34:01 +0000 2012       es                 1   
1        Mon Oct 29 19:11:23 +0000 2012       es                 1   
2        Mon Oct 29 18:11:08 +0000 2012       es                 1   
3        Mon Oct 29 19:15:33 +0000 2012       en                 1   
4        Mon Oct 29 20:46:02 +0000 2012       en                 1   
...                                 ...      ...               ...   
14478  Tue Mar 11 03: 48: 36 +0000 2014       en                 0   
14479  Mon Mar 10 18: 09: 26 +0000 2014       en                 1   
14480  Mon Mar 10 10: 59: 45 +0000 2014       en                 0   
14481  Sun Mar 09 20: 07: 10 +0000 2014       it                 1   
14482  Sun Mar 09 16: 36: 09 +0000 2014       en                 0   

       significant_real  misspellings  news_company_found  num_emoji  \
0                     0             0                   0          0   
1                     0             1                   0          0   
2                     0             1                   0          0   
3                     0             0                   0          0   
4                     0             2                   0          1   
...                 ...           ...                 ...        ...   
14478                 0             4                   0          0   
14479                 0             2                   0          0   
14480                 0             3                   0          0   
14481                 0             3                   0          0   
14482                 0             3                   0          0   

       num_hashtags  profane  retweet  label  
0                 1        0        0      0  
1                 0        0        1      0  
2                 2        0        0      0  
3                 2        1        0      0  
4                 4        0        0      0  
...             ...      ...      ...    ...  
14478             0        0        0      0  
14479             0        0        0      0  
14480             1        0        0      0  
14481             0        0        0      0  
14482             1        0        0      0  

[14483 rows x 13 columns]

In [24]:
df_train = df_train[['text', 'imageIds', 'timestamp', 'language', 'significant_fake', 'significant_real', 'profane', 'misspellings', 'num_emoji', 'num_hashtags', 'news_company_found', 'retweet', 'label']]
df_train

text        imageIds  \
0      remember movie Day Tomorrow reminds happening ...  sandyA_fake_46   
1      Look Sandy NY Tremendous image hurricane Looks...  sandyA_fake_09   
2      Good photo Hurricane Sandy reminds movie Indep...  sandyA_fake_09   
3                                Scary shit hurricane NY  sandyA_fake_29   
4      fave place world nyc hurricane sandy statueofl...  sandyA_fake_15   
...                                                  ...             ...   
14478         slaps TweetDeck PigFish http co pyHcJn0jwA      pigFish_01   
14479  New Species Fish found Brazil Really good Phot...      pigFish_01   
14480                    call pigFISH http co 4Bml62OD15      pigFish_01   
14481         Pigfish shark pork fish http co hQzWGhyDef      pigFish_01   
14482        decide fish meat Pigfish http co 5JBtF54cmg      pigFish_01   

                              timestamp language  significant_fake  \
0        Mon Oct 29 22:34:01 +0000 2012       es                 1   
1        Mon Oct 29 19:11:23 +0000 2012       es                 1   
2        Mon Oct 29 18:11:08 +0000 2012       es                 1   
3        Mon Oct 29 19:15:33 +0000 2012       en                 1   
4        Mon Oct 29 20:46:02 +0000 2012       en                 1   
...                                 ...      ...               ...   
14478  Tue Mar 11 03: 48: 36 +0000 2014       en                 0   
14479  Mon Mar 10 18: 09: 26 +0000 2014       en                 1   
14480  Mon Mar 10 10: 59: 45 +0000 2014       en                 0   
14481  Sun Mar 09 20: 07: 10 +0000 2014       it                 1   
14482  Sun Mar 09 16: 36: 09 +0000 2014       en                 0   

       significant_real  profane  misspellings  num_emoji  num_hashtags  \
0                     0        0             0          0             1   
1                     0        0             1          0             0   
2                     0        0             1          0             2   
3                     0        1             0          0             2   
4                     0        0             2          1             4   
...                 ...      ...           ...        ...           ...   
14478                 0        0             4          0             0   
14479                 0        0             2          0             0   
14480                 0        0             3          0             1   
14481                 0        0             3          0             0   
14482                 0        0             3          0             1   

       news_company_found  retweet  label  
0                       0        0      0  
1                       0        1      0  
2                       0        0      0  
3                       0        0      0  
4                       0        0      0  
...                   ...      ...    ...  
14478                   0        0      0  
14479                   0        0      0  
14480                   0        0      0  
14481                   0        0      0  
14482                   0        0      0  

[14483 rows x 13 columns]

In [37]:
def pull_features(data_frame):
    features = []
    for i in range(len(data_frame)):
        temp = []
        temp.append(data_frame.iloc[i]['significant_fake'])
        temp.append(data_frame.iloc[i]['significant_real'])
        temp.append(data_frame.iloc[i]['profane'])
        temp.append(data_frame.iloc[i]['misspellings'])
        temp.append(data_frame.iloc[i]['num_emoji'])
        #temp.append(data_frame.iloc[i]['num_hashtags'])
        temp.append(data_frame.iloc[i]['news_company_found'])
        features.append(temp)
    features = np.array(features)
    return features

In [38]:
X_train_features = pull_features(df_train)
X_test_features = pull_features(df_test)

In [27]:
X_train_features.shape

(14483, 7)

In [41]:
text_input = tf.keras.layers.Input(shape=(6,), name='features')
text_embed = tf.keras.layers.Embedding(79 + 1, 50, input_length=6, name='article_body_embedding')(text_input)
text_conv = tf.keras.layers.Conv1D(256, 6, name='article_body_conv')(text_embed)
text_pool = tf.keras.layers.GlobalMaxPool1D(name='article_body_pooling')(text_conv)
#concat = tf.keras.layers.concatenate([text_pool])
dense_100 = tf.keras.layers.Dense(30, activation='relu')(text_pool)
dense_50 = tf.keras.layers.Dense(10, activation='relu')(dense_100)
out_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_50)
model = tf.keras.models.Model(inputs=[text_input], outputs=out_layer)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (InputLayer)        [(None, 6)]               0         
_________________________________________________________________
article_body_embedding (Embe (None, 6, 50)             4000      
_________________________________________________________________
article_body_conv (Conv1D)   (None, 1, 256)            77056     
_________________________________________________________________
article_body_pooling (Global (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 30)                7710      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [42]:
y_train = df_train['label'].values
y_test = df_test['label'].values
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='max', restore_best_weights=True)

history = model.fit(X_train_features, y_train, epochs=100, batch_size=128, validation_data=(X_test_features, y_test), callbacks=[callback])

Train on 14483 samples, validate on 3781 samples
Epoch 1/100
14483/14483 [==============================] - 1s 61us/sample - loss: 0.5166 - acc: 0.7665 - val_loss: 0.6554 - val_acc: 0.7720
Epoch 2/100
14483/14483 [==============================] - 1s 50us/sample - loss: 0.4942 - acc: 0.7837 - val_loss: 0.6142 - val_acc: 0.6699
Epoch 3/100
14483/14483 [==============================] - 1s 53us/sample - loss: 0.4956 - acc: 0.7826 - val_loss: 0.5822 - val_acc: 0.7675
Epoch 4/100
14483/14483 [==============================] - 1s 52us/sample - loss: 0.4938 - acc: 0.7837 - val_loss: 0.6548 - val_acc: 0.6707
Epoch 5/100
14483/14483 [==============================] - 1s 50us/sample - loss: 0.4936 - acc: 0.7855 - val_loss: 0.6430 - val_acc: 0.7707
Epoch 6/100
14483/14483 [==============================] - 1s 59us/sample - loss: 0.4951 - acc: 0.7829 - val_loss: 0.6264 - val_acc: 0.7686


In [43]:
preds = model.predict(X_test_features)
model.evaluate(X_test_features, y_test)


[0.65543099035822, 0.77201796]

In [32]:
preds.shape
#y_test = np.array(y_test)
print(preds.shape)
tf.reshape(preds, [-1, 1])
tf.reshape(y_test, [-1, 1])
print(y_test.shape)

(3781, 1)
(3781,)


In [44]:
preds
y_pred = []
for i in range(len(preds)):
    if(preds[i]<0.5):
        y_pred.append(0)
    else:
        y_pred.append(1)
    
y_pred = np.array(y_pred)



In [45]:
import sklearn.metrics as skl_m

f1 = skl_m.f1_score(y_test, y_pred)
f1

0.7070020394289599